In [1]:
#%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
import matplotlib.cm     as cm
import matplotlib.collections as mc
import numpy as np
import cv2

from larlite import larlite as fmwk
from ROOT import fcllite

cfg_mgr = fcllite.ConfigManager()
cfg_mgr.AddCfgFile("../mac/ClusterHoughPSet.fcl")

config = cfg_mgr.Config()
pset = config.get_pset("ClusterHough"); 

my_proc = fmwk.ana_processor()
my_proc.add_input_file("/Users/vgenty/Desktop/larlite_reco2d_000.root")

my_proc.set_io_mode(fmwk.storage_manager.kBOTH)                                                                                                                     
my_proc.set_output_file("aho.root")
my_proc.set_data_to_write(fmwk.data.kHit,'gaushit')                                                                                                                 
my_proc.set_data_to_write(fmwk.data.kCluster,'hullcluster')                                                                                                         
my_proc.set_data_to_write(fmwk.data.kAssociation,'hullcluster')                 

my_unit  = fmwk.HitImageManager()
my_aho   = fmwk.AlgoImageMaker()
my_aho2  = fmwk.AlgoClusterConvex(pset)

my_unit.SetImageMaker(my_aho)
my_unit.SetClusterMaker(my_aho2)
my_proc.add_process(my_unit)    

0.0

In [2]:
my_proc.process_event()

True

In [3]:
plane = 2
n_hulls = my_aho2.possible_break_num(plane)
print n_hulls

4


In [12]:
idx = 0
r = my_aho2.possible_break(plane,idx)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
r = cv2.dilate(r,kernel,iterations = 2)
r = cv2.blur(r,(5,5))
ret,r = cv2.threshold(r,0,255,cv2.THRESH_BINARY)

_, contours0, hierarchy = cv2.findContours( r.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = [cv2.approxPolyDP(cnt, 0.1, True) for cnt in contours0]
print 'I found...%d contours' % len(contours)

contours.sort(key = len)
ctor = -1
print len(contours[ctor])

x1  = []
y1  = []
xy1 = []

for i in contours[ctor]:
    x1.append(i[0][0])
    y1.append(i[0][1])
    xy1.append( [i[0][0], i[0][1]] )

x1.append(x1[0])
y1.append(y1[0])
xy1.append([x1[0], y1[0]])

chull = cv2.convexHull(np.array(xy1),returnPoints = False)
chull = chull[:,0].tolist()
chull.append(chull[0])
defects = cv2.convexityDefects(np.array(xy1),np.array(chull))

x2 = []
y2 = []
for i in chull:
    x2.append(x1[i])
    y2.append(y1[i])
    
fig,ax = plt.subplots(figsize=(10,6))

ax.plot(x1,y1,color='blue',lw=2) 
ax.plot(x2,y2,color='red',lw=2) 

lines = []
px = []
py = []

for i in xrange(defects.shape[0]):
    s,e,f,d = defects[i,0]
    start = (x1[s],y1[s])
    end =   (x1[e],y1[e])
    far =   (x1[f],y1[f])
    lines.append([start, far])
    lines.append([far, end])
    px.append(x1[f])
    py.append(y1[f])

lc = mc.LineCollection(lines, linewidths=10,color='green',alpha=0.7)
ax.imshow(r,cmap = cm.Greys,interpolation='none')
ax.add_collection(lc)
ax.plot(px,py,'o',color='purple')
ax.autoscale()
plt.show()

I found...6 contours
1034


In [10]:
for i in xrange(len(defects)):
    print defects[i][0][3]/256.0

57.1328125
0.4453125
8.9296875
26.7890625
0.8203125
0.84765625
6.0
38.390625
0.4453125
24.5078125
6.32421875


In [20]:
defects.shape[0]

2